## Introduction of GEE and NDVI for AOE
#TD

## Setting your toolbox

In [1]:
# !pip install earthengine-api
# !pip install folium 
# !pip install geehydro
# !pip install proplot
# !pip install ipygee

In [92]:
import ee 
import folium
import geehydro
import xarray as xr

from datetime import datetime as dt
from IPython.display import Image
import pandas as pd
#import proplot as plot 
import matplotlib.pyplot as plt 

from ipygee import *
from pandas.plotting import register_matplotlib_converters
import branca.colormap as cm
import folium.plugins 
import branca
from folium.plugins import * 

## Initialize the connection to the server

In [3]:
# ee.Authenticate()
ee.Initialize()

## Select a region in the world

In [ ]:
# the Korea Seoul .
Himalaya_map = folium.Map(location=[28.219812566703645,85.59529116364365,], zoom_start=10,tiles="Stamen Terrain")
Himalaya_map

## Setting the Area of Interest (AOI)

In [100]:
# Himalaya_AOI = ee.Geometry.Rectangle([126.5,37.0,
#                                    127.5,37.9])   #xlo", "ylo", "xhi" and "yhi" 
# # Seoul_AOI = ee.Geometry.Point([126.9780,37.5665]).buffer(20000)
# # help(ee.Geometry.Rectangle)
Himalaya_AOI = ee.Geometry.Polygon([[70.37980734782377,34.20014926719452],
[70.86320578532377,34.20014926719452],
 [72.40129172282377,34.05463899658766],
 [73.41203391032377,33.360057261802915],
 [74.95011984782377,32.326294617152875],
 [76.66398703532377,31.205448585010295],
 [78.24601828532377,30.071161703174734],
 [79.82804953532377,29.307644833926915],
 [81.45402609782377,28.306484308226114],
 [83.65129172282377,27.607788110834946],
 [87.47453391032377,27.178594650769533],
 [88.48527609782377,26.982957823282142],
 [91.95695578532377,26.826202971177672],
 [96.08781516032377,28.267786780126247],
 [96.08781516032377,29.000621559555128],
 [95.29679953532377,29.728297185240383],
 [94.15422141032377,29.192618684440546],
 [92.74797141032377,28.15160989157106],
 [91.42961203532377,28.074088505068868],
 [89.36418234782377,27.99651113483418],
 [86.90324484782377,28.34516776469134],
 [84.00285422282377,29.115862806544595],
 [79.08097922282377,31.318142578838632],
 [76.40031516032377,33.24987411988282],
[75.30168234782377,34.56282699752331],
 [73.76359641032377,35.13983300015285],
 [72.04972922282377,35.748451422480656],
 [70.46769797282377,35.85537617379362],
 [69.85246359782377,34.30911746948516],
 [70.37980734782377,34.20014926719452]],   

)
Himalaya_map.addLayer(Himalaya_AOI)
Himalaya_map


## The Landsat 8 Collection

In [6]:
### Landset NDVI bands

In [7]:
# NDVIL = ee.ImageCollection('MODIS/006/MOD13Q1').filterDate(ee.DateRange('2016-01-01',
#                                                                         '2019-12-01'))
NDVIL = ee.ImageCollection("LANDSAT/LC08/C01/T1_32DAY_NDVI").filterDate(ee.DateRange('2013-04-07',
                                                                        '2022-01-01'))
ndvi = NDVIL.select('NDVI')


In [8]:
# landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# # choose dates
# landsat = landsat.filterDate('2019-07-01','2019-12-01')
# # filter area
# landsat_AOI = landsat.filterBounds(Seoul_AOI)


In [ ]:
landsat_AOI = ndvi.filterBounds(Himalaya_AOI)
aa=landsat_AOI.reduce(ee.Reducer.median())
aa.getInfo()


Exporting map with extent

In [10]:
# the best wat to downlaod tiff file of AOI regiion 
import requests
import ee

# img = ee.Image('COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')
# region = ee.Geometry.BBox(-122.0859, 37.0436, -122.0626, 37.0586)

# Multi-band GeoTIFF file.
url = aa.getDownloadUrl({
    # 'bands': ['B3', 'B8', 'B11'],
    'region': Himalaya_AOI,
    'scale': 2000,
    'format': 'GEO_TIFF'
})
response = requests.get(url)
with open('multi_band_median.tif', 'wb') as fd:
  fd.write(response.content)

In [11]:
# pip install geemap

In [12]:
# # Load Landsat 8 image collection Different bands 
import geemap
import time
aoi=Himalaya_AOI
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(aoi) \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER')

# Clip the image collection to the AOI
collection = collection.map(lambda image: image.clip(aoi))

# Plot the image collection
Map = geemap.Map()
Map.centerObject(aoi, zoom=12)
Map.addLayer(collection.median(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Landsat 8')
Map.addLayer(aoi, {'color': 'green'}, 'AOI')
Map.addLayerControl()
Map


Map(center=[30.591255714068318, 82.2792850527308], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# from google.colab import files
# import time
# image=aa

# export_params = {
#   'image': image,
#   'description': 'image_to_drive_example',
#   'folder': 'GEE_exports',
#   'scale': 30,
#   'region': image.geometry().bounds(),
#   'fileFormat': 'GeoTIFF'
# }

# # Create the export task
# task = ee.batch.Export.image.toDrive(**export_params)

# task.start()

# # Wait for the export task to complete
# while task.status()['state'] in ['READY', 'RUNNING']:
#     print('Export task is', task.status()['state'].lower())
#     time.sleep(10)

# # Check if the export task completed successfully
# if task.status()['state'] == 'COMPLETED':
#     print('Export task completed successfully!')
# else:
#     print('Export task failed or was cancelled.')
# # files.download('/content/drive/My Drive/GEE_outputs/mean_image.tif')

In [15]:
#scaling image
def scale_factor(image):
  # scale factor for the MODIS MOD13Q1 product
  return image.multiply(1).copyProperties(image, ['system:time_start'])

In [16]:
# mapping function to multiply by the scale factor and number of layers 
scaled_ndvi = ndvi.map(scale_factor)
scaled_ndvi.size().getInfo()
# ndvi.size().getInfo()

105

In [17]:
# Himalaya MOD13Q1 NDVI time series
Himalaya_AOI_ndvi = chart.Image.series(**{'imageCollection': scaled_ndvi,     # can chagne 'scaled_ndvi'
                                   'region': Himalaya_AOI,
                                   'reducer': ee.Reducer.mean(),
                                   'scale': 5000,
                                   'xProperty': 'system:time_start'
                                  })
Himalaya_AOI_ndvi.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [18]:
Himalaya_AOI_ndvi.dataframe.to_csv('Himalaya_AOI_ndvi.csv')
df=Himalaya_AOI_ndvi.dataframe
df

,NDVI
2013-04-11,0.229505
2013-05-13,0.246007
2013-06-14,0.199101
2013-07-16,0.213923
2013-08-17,0.300335
...,...
2021-08-13,0.260188
2021-09-14,0.328243
2021-10-16,0.353388
2021-11-17,0.264412


In [19]:
# fig, ax = plot.subplots(figsize=(7, 3), tight=True)
# ax.plot(Himalaya_AOI_ndvi.dataframe.index, Himalaya_AOI_ndvi.dataframe['NDVI'],
#         color='forest green', marker='o')
# plot.show()

In [103]:
## mean NDVI in the Himalaya region 
bb=Himalaya_map.addLayer(scaled_ndvi.mean().clip(Himalaya_AOI),
             vis_params={'min': 0.00001,
                         'max': 1,
                         'palette': ['red', 'yellow', 'green']})

colormap = cm.LinearColormap(colors=['red','green'], index=[-0.9,0.6],vmin=-0.84,vmax=0.6)
Himalaya_map.add_child(colormap)
minimap = plugins.MiniMap()   # for minmap of survey area
Himalaya_map.add_child(minimap)
# # Add a minimap with a scale bar
# MM = MiniMap(toggle_display=True, minimized=True, position='bottomleft', 
#                   zoom_level_offset=-6, width=150, height=150, metric=True, imperial=False)
# Himalaya_map.add_child(MM)
display(Himalaya_map)

In [65]:

Himalaya_map.save("filename.tif")
Himalaya_map.save("filename1.html")



In [ ]:
# #Export the image, specifying the CRS, transform, and region.
# task=ee.batch.Export.image.toDrive(
#   image= ndvi,
#   # image=aa,
#   scale=1000,
#   region=Himalaya_AOI,
#   description= 'imageToDriveExample_transform',
#   fileNamePrefix='my_export_lyon1',
#   # crs='EPSG:4326',
#   fileFormat='GeoTIFF'
   
# );
# task.start()
# #checking the status for file saviing 
# while True:
#   cc=task.status()
#   if cc['state']=='COMPLETED':
#     break



In [26]:
# pwd


In [ ]:
landsat_AOI = ndvi.filterBounds(Himalaya_AOI)
aa=landsat_AOI.reduce(ee.Reducer.median())
aa.getInfo()

link = aa.getDownloadURL({   ## aa is the file that we hd cropted
    'scale': 1000, 
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': Himalaya_AOI})
print(link)

In [ ]:
Himalaya_AOI.getInfo()['coordinates']

In [ ]:
# define your non-rectangular polygon
aoi = Himalaya_AOI.getInfo()['coordinates']

# load the image
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').clip(aoi)

# clip the image to the polygon boundary
clipped = image.clip(aoi)

# export the clipped image to Google Drive
task = ee.batch.Export.image.toDrive(image=clipped, description='clipped_image', folder='my_folder', scale=30)
task.start()
#checking the status for file saviing 
while True:
  cc=task.status()
  if cc['state']=='COMPLETED':
    break

## downloading only clip regiion

In [81]:
# Load the image and polygon
# Clip the image to the polygon
clipped_image = aa.clip(Himalaya_AOI)

# Get the download URL for the clipped image
url = clipped_image.getDownloadURL({
    'name': 'clipped_image',
    'scale': 2000,
    'region': Himalaya_AOI
})
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/994b3fdb44f57dc5d1ef3ef052c34028-a24149ae0e472706b38517a3120910e1:getPixels


In [ ]:
Himalaya_AOI.getInfo()['coordinates']

In [34]:
# import os
# import folium

# m = folium.Map([28,85], zoom_start=1, tiles="stamentoner")
# merc = os.path.join("data", "/content/filename.png")


# if not os.path.isfile(merc):
#     print(f"Could not find {merc}")
# else:
#     img = folium.raster_layers.ImageOverlay(
#         name="filename",
#         image=merc,
#         bounds=[[26.5, 69.85], [36,96]],
#         # bounds=Himalaya_AOI,
#         # opacity=0.6,
#         interactive=True,
#         cross_origin=False,
#         zindex=1,
#     )

#     folium.Popup("I am an image").add_to(img)

#     img.add_to(m)
#     folium.LayerControl().add_to(m)

# m

In [ ]:
## Temperature data

In [35]:
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2013-03-18', '2022-12-29');
dataset.size().getInfo()


1495387

In [ ]:
#scaling image
def scale_factor(image):
  # scale factor for the MODIS MOD13Q1 product
  # return image.select('ST_B10').multiply(1).copyProperties(image, ['system:time_start'])
 return  image.select('ST_B.*').multiply(0.00341802).add(149.0)

In [ ]:
scaled_temp = dataset.map(scale_factor)
scaled_temp.size().getInfo()

## Precipitatio in AOI 

In [ ]:
# Load the precipitation dataset
precipitation = ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('prcp')

# Filter the precipitation dataset for the AOI and time range
precipitation = precipitation.filterBounds(Himalaya_AOI).filterDate(ee.DateRange('2013-04-07',
                                                                        '2022-01-01'))

Temperatue in AOI

In [ ]:
# Load the image collection for the location and time period
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
  .filterBounds(Himalaya_AOI)\
  .filterDate('2013-04-07','2022-01-01')

# Select a single image from the collection
image = collection.first()

In [ ]:
# Define the region of interest
#aoi = ee.Geometry.Rectangle([min_longitude, min_latitude, max_longitude, max_latitude])

# Load MODIS NDVI data for the region and time period of interest
collection = ee.ImageCollection('MODIS/006/MOD13Q1').filterDate('2019-01-01', '2019-12-31').filterBounds(Himalaya_AOI)

# Calculate the mean NDVI for the region and time period
ndvi_mean = collection.select('NDVI').mean().clip(Himalaya_AOI)

# Print the result
print('Mean NDVI:', ndvi_mean.getInfo())


In [ ]:
# Reduce the NDVI data to a single value for the region
ndvi_result = ndvi_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=Himalaya_AOI, scale=250)

# Convert the result to a pandas DataFrame
ndvi_df = pd.DataFrame(ndvi_result.getInfo(), index=['NDVI'])

# Print the DataFrame
print(ndvi_df)


Export in Pandas dataframe

In [ ]:
# the names of each Landsat 8 band
# https://www.usgs.gov/media/images/landsat-8-band-designations
# https://www.usgs.gov/landsat-missions/landsat-8-9-surface-reflectance-quality-assessment

landsat_AOI.first().bandNames().getInfo()